In [1]:
import pandas as pd

In [2]:
trips_path = "gtfs_static/trips.txt"
trips = pd.read_csv(trips_path)

routes_path = "gtfs_static/routes.txt"
routes = pd.read_csv(routes_path)

shapes_path = "gtfs_static/shapes.txt"
shapes = pd.read_csv(shapes_path)

# stop_times_path = "gtfs_static/stop_times.txt"
# stop_times = pd.read_csv(stop_times_path)

stops_path = "gtfs_static/stops.txt"
stops = pd.read_csv(stops_path)


In [3]:
import pandas as pd

def gtfs_shapes_to_geojson(df):
    """
    Convert a GTFS shapes DataFrame into a GeoJSON object.

    Parameters:
        df (pandas.DataFrame): DataFrame with GTFS shapes data having columns:
            - shape_id
            - shape_pt_lat
            - shape_pt_lon
            - shape_pt_sequence
            - shape_dist_traveled

    Returns:
        dict: A GeoJSON-like dictionary with key "routes" mapping to a FeatureCollection.
    """
    features = []
    
    # Group the DataFrame by shape_id
    for shape_id, group in df.groupby("shape_id"):
        # Sort points by the sequence
        group_sorted = group.sort_values("shape_pt_sequence")
        # GeoJSON requires coordinates as [longitude, latitude]
        coordinates = group_sorted[['shape_pt_lon', 'shape_pt_lat']].values.tolist()
        
        # Create a GeoJSON Feature for the current route
        feature = {
            "type": "Feature",
            "properties": {"shape_id": shape_id},
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            }
        }
        features.append(feature)
    
    # Build the final GeoJSON object wrapped in a "routes" key
    geojson = {
        "routes": {
            "type": "FeatureCollection",
            "features": features
        }
    }
    
    return geojson


In [4]:
def get_shape_by_route_id(geojson, route_id):
    """
    Given a GeoJSON dictionary with a "routes" key and a route_id,
    return the geometry for that route.

    Parameters:
        geojson (dict): A dictionary with a "routes" key containing a FeatureCollection.
        route_id (str or int): The route ID to query for.

    Returns:
        dict or None: The geometry of the matching route as a GeoJSON object, or None if not found.
    """
    features = geojson.get("routes", {}).get("features", [])
    for feature in features:
        if feature.get("properties", {}).get("shape_id") == route_id:
            return feature.get("geometry")
    return None

In [5]:
import json


In [6]:
translink_shapes = gtfs_shapes_to_geojson(shapes)

In [21]:
# translink_shapes

In [24]:
get_shape_by_route_id(translink_shapes, 6621)

In [8]:
with open("gtfs_processed/shapes.geojson", "w") as f:
    json.dump(translink_shapes, f, indent=2)

In [9]:
get_shape_by_route_id(translink_shapes, 6619)

In [10]:
# translink_shapes['routes']

In [11]:
import h3

In [12]:
for res in range(7, 11):
    shapes[f'h3_{res}'] = shapes.apply(lambda row: h3.geo_to_h3(row['shape_pt_lat'], row['shape_pt_lon'], res), axis=1)

In [13]:
shapes.to_csv("gtfs_processed/shapes_enriched.csv", index=False)

In [14]:
shapes

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,h3_7,h3_8,h3_9,h3_10
0,296955,49.234553,-123.185890,1,0.0000,8728de8c1ffffff,8828de8c19fffff,8928de8c197ffff,8a28de8c194ffff
1,296955,49.234719,-123.170253,10,1.1627,8728de8c1ffffff,8828de8c57fffff,8928de8c1d3ffff,8a28de8c1d27fff
2,296955,49.277013,-123.132288,100,8.6575,8728de8dcffffff,8828de8d89fffff,8928de8dc2fffff,8a28de8dc2dffff
3,296955,49.277089,-123.132228,101,8.6670,8728de8dcffffff,8828de8d89fffff,8928de8dc2fffff,8a28de8dc2dffff
4,296955,49.277669,-123.131347,102,8.7580,8728de8d8ffffff,8828de8d89fffff,8928de8d893ffff,8a28de8d8977fff
...,...,...,...,...,...,...,...,...,...
156212,301880,49.260572,-122.774042,96,8.4679,8728dec45ffffff,8828dec455fffff,8928dec4427ffff,8a28dec455a7fff
156213,301880,49.260671,-122.773951,97,8.4808,8728dec45ffffff,8828dec455fffff,8928dec4427ffff,8a28dec455a7fff
156214,301880,49.260780,-122.773863,98,8.4945,8728dec45ffffff,8828dec455fffff,8928dec455bffff,8a28dec455a7fff
156215,301880,49.260953,-122.773816,99,8.5140,8728dec45ffffff,8828dec455fffff,8928dec455bffff,8a28dec455a7fff


In [15]:
routes

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,10232,TL,256,Whitby Estate/Park Royal/Spuraway,NaN,3,NaN,NaN,NaN
1,11201,TL,033,29th Avenue Station/UBC,NaN,3,NaN,NaN,NaN
2,11692,TL,364,Langley Centre/Scottsdale,NaN,3,NaN,NaN,NaN
3,11693,TL,388,22nd St Station/Carvolth Exchange,NaN,3,NaN,NaN,NaN
4,11696,TL,609,South Delta Exchange/Ladner Exchange,NaN,3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
234,8286,TL,370,Cloverdale/Willowbrook,NaN,3,NaN,NaN,NaN
235,8289,TL,865,Samuel Robertson,NaN,3,NaN,NaN,NaN
236,8290,TL,341,Guildford/Newton Exchange,NaN,3,NaN,NaN,NaN
237,9745,TL,418,Kingswood/22nd St Station,NaN,3,NaN,NaN,NaN


In [16]:
shapes

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,h3_7,h3_8,h3_9,h3_10
0,296955,49.234553,-123.185890,1,0.0000,8728de8c1ffffff,8828de8c19fffff,8928de8c197ffff,8a28de8c194ffff
1,296955,49.234719,-123.170253,10,1.1627,8728de8c1ffffff,8828de8c57fffff,8928de8c1d3ffff,8a28de8c1d27fff
2,296955,49.277013,-123.132288,100,8.6575,8728de8dcffffff,8828de8d89fffff,8928de8dc2fffff,8a28de8dc2dffff
3,296955,49.277089,-123.132228,101,8.6670,8728de8dcffffff,8828de8d89fffff,8928de8dc2fffff,8a28de8dc2dffff
4,296955,49.277669,-123.131347,102,8.7580,8728de8d8ffffff,8828de8d89fffff,8928de8d893ffff,8a28de8d8977fff
...,...,...,...,...,...,...,...,...,...
156212,301880,49.260572,-122.774042,96,8.4679,8728dec45ffffff,8828dec455fffff,8928dec4427ffff,8a28dec455a7fff
156213,301880,49.260671,-122.773951,97,8.4808,8728dec45ffffff,8828dec455fffff,8928dec4427ffff,8a28dec455a7fff
156214,301880,49.260780,-122.773863,98,8.4945,8728dec45ffffff,8828dec455fffff,8928dec455bffff,8a28dec455a7fff
156215,301880,49.260953,-122.773816,99,8.5140,8728dec45ffffff,8828dec455fffff,8928dec455bffff,8a28dec455a7fff


In [17]:
trips

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,6613,1,14252539,3 Main/To Waterfront Station,NaN,0,2117693.0,296959,0,1
1,6613,1,14252540,3 Main/To Waterfront Station,NaN,0,2117692.0,296959,0,1
2,6613,1,14252541,3 Main/To Waterfront Station,NaN,0,2117691.0,296959,0,1
3,6613,1,14252542,3 Main/To Waterfront Station,NaN,0,2117690.0,296959,0,1
4,6613,1,14252543,3 Main/To Waterfront Station,NaN,0,2117695.0,296959,0,1
...,...,...,...,...,...,...,...,...,...,...
61454,13686,3,14514283,Canada Line To YVR-Airport,NaN,1,2135831.0,298068,0,1
61455,13686,3,14514284,Canada Line To Richmond-Brighouse,NaN,1,2135832.0,298069,0,1
61456,13686,3,14514285,Canada Line To YVR-Airport,NaN,1,2135835.0,298068,0,1
61457,13686,3,14514286,Canada Line To Richmond-Brighouse,NaN,1,2135836.0,298069,0,1


In [18]:
for res in range(7, 11):
    stops[f'h3_{res}'] = stops.apply(lambda row: h3.geo_to_h3(row['stop_lat'], row['stop_lon'], res), axis=1)

In [19]:
stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,wheelchair_boarding,h3_7,h3_8,h3_9,h3_10
0,1,50001.0,Westbound Davie St @ Bidwell St,NaN,49.286458,-123.140424,BUS ZN,NaN,0,NaN,1,8728de8d8ffffff,8828de8d8bfffff,8928de8d8a7ffff,8a28de8d8a57fff
1,10000,59326.0,Northbound No. 5 Rd @ McNeely Dr,NaN,49.179962,-123.091490,BUS ZN,NaN,0,NaN,1,8728de8f4ffffff,8828de8f41fffff,8928de8f48fffff,8a28de8f48dffff
2,10001,59324.0,Northbound No. 5 Rd @ Woodhead Rd,NaN,49.182670,-123.091448,BUS ZN,NaN,0,NaN,1,8728de8f4ffffff,8828de8f41fffff,8928de8f417ffff,8a28de8f402ffff
3,10002,59323.0,Southbound No. 5 Rd @ Cambie Rd,NaN,49.184252,-123.091627,BUS ZN,NaN,0,NaN,1,8728de8f4ffffff,8828de8f41fffff,8928de8f407ffff,8a28de8f4057fff
4,10003,59325.0,Southbound No. 5 Rd @ Woodhead Rd,NaN,49.182051,-123.091659,BUS ZN,NaN,0,NaN,1,8728de8f4ffffff,8828de8f41fffff,8928de8f417ffff,8a28de8f4157fff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8838,99955,NaN,Maple Meadows Station,NaN,49.216513,-122.666210,WCE3Z,NaN,1,NaN,1,8728dec29ffffff,8828dec295fffff,8928dec2947ffff,8a28dec2945ffff
8839,99956,NaN,Port Haney Station,NaN,49.212202,-122.605240,WCE3Z,NaN,1,NaN,1,8728dec05ffffff,8828dec2adfffff,8928dec2adbffff,8a28dec2ad97fff
8840,99957,NaN,Mission City Station,NaN,49.133694,-122.304898,WCE4Z,NaN,1,NaN,1,8728dedb0ffffff,8828dedb01fffff,8928dedb013ffff,8a28dedb0137fff
8841,99958,NaN,Lonsdale Quay Station,NaN,49.310142,-123.083309,ZN 2,NaN,1,NaN,1,8728de129ffffff,8828de129bfffff,8928de129a3ffff,8a28de129a07fff


In [20]:
stops.to_csv("gtfs_processed/stops_enriched.csv", index=False)